In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

# Configurar Pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [2]:
print("Ajuste de perfil")
with open('user_id.json', 'r') as f:
    user_id = json.load(f)['ID_Usuario']
#print(user_id)

# Leer el archivo CSV
dfusuarios = pd.read_csv('usuarios.csv')

Ajuste de perfil


In [3]:
def MostrarVistaGenDeseados():
    with output_vista:
        output_vista.clear_output()  # Limpiar el contenido anterior
        display(vista_deseados)

def MostrarVistaGenNoDeseados():
    with output_vista:
        output_vista.clear_output()  # Limpiar el contenido anterior
        display(vista_no_deseados)
        inicializar_botones_no_deseados()

In [4]:
def boton_clicado1(b):
    genero = b.description
    if estado_generos_deseados[genero] is None:
        valores = [v for v in estado_generos_deseados.values() if v is not None]
        nuevo_valor = max(valores, default=0) + 1 if valores else 1
        if len(valores) < 5:
            estado_generos_deseados[genero] = nuevo_valor
            b.style.button_color = 'lightgreen'
    else:
        orden = estado_generos_deseados[genero]
        for key, value in estado_generos_deseados.items():
            if value and value > orden:
                estado_generos_deseados[key] = value - 1
        estado_generos_deseados[genero] = None
        b.style.button_color = None
    mostrar_seleccionados1()
    actualizar_boton_continuar1()

def mostrar_seleccionados1():
    with output_deseados2:
        clear_output()
        seleccionados = sorted([(g, v) for g, v in estado_generos_deseados.items() if v is not None], key=lambda x: x[1])
        if seleccionados:
            print("Géneros seleccionados por orden de preferencia:")
            for i, (genero, orden) in enumerate(seleccionados, start=1):
                print(f"{i}. {genero}")
        else:
            print("No hay géneros seleccionados.")

def actualizar_boton_continuar1():
    valores = [v for v in estado_generos_deseados.values() if v is not None]
    btn_continuar1.disabled = not valores

def continuar1():
    global dfusuarios
    index_usuario = dfusuarios[dfusuarios['ID_Usuario'] == user_id].index[0]
    valores = sorted([(g, v) for g, v in estado_generos_deseados.items() if v is not None], key=lambda x: x[1])
    for i, (genero, orden) in enumerate(valores, start=1):
        col_index = generos.index(genero) + 7
        dfusuarios.iat[index_usuario, col_index] = 1 - 0.05 * (i - 1)
    dfusuarios.to_csv('usuarios.csv', index=False)
    MostrarVistaGenNoDeseados()

# Función para manejar el evento de click en los botones de géneros no deseados
def boton_clicado2(b):
    genero = b.description
    if estado_generos_no_deseados[genero] is None:
        valores = [v for v in estado_generos_no_deseados.values() if v is not None]
        if len(valores) < 5:
            estado_generos_no_deseados[genero] = True
            b.style.button_color = 'red'
        else:
            print("Máximo de 5 géneros no deseados alcanzado")
    else:
        estado_generos_no_deseados[genero] = None
        b.style.button_color = None
    actualizar_boton_continuar2()

def actualizar_boton_continuar2():
    valores = [v for v in estado_generos_no_deseados.values() if v is not None]
    btn_continuar2.disabled = not valores

def continuar2():
    global user_id
    index_usuario = dfusuarios[dfusuarios['ID_Usuario'] == user_id].index[0]
    valores = [g for g, v in estado_generos_no_deseados.items() if v is not None]
    for genero in valores:
        col_index = generos.index(genero) + 7
        dfusuarios.iat[index_usuario, col_index] = 0
    dfusuarios.to_csv('usuarios.csv', index=False)
    print("Géneros no deseados actualizados")
    with open('user_id.json', 'w') as f:
        json.dump({'ID_Usuario': user_id}, f)
    with output_vista:
        # Redirigir al segundo notebook
        display(Javascript('window.location.href = "ajustepeliculas.ipynb";'))

# Función para inicializar los botones de géneros no deseados
def inicializar_botones_no_deseados():
    for genero, boton in botones_no_deseados.items():
        if estado_generos_deseados[genero] is not None:
            # Desactivar el botón si el género ya fue seleccionado como deseado
            boton.disabled = True
            boton.style.button_color = 'lightgreen'
        else:
            # Restablecer el estado normal del botón si no fue seleccionado como deseado
            boton.disabled = False
            boton.style.button_color = None

def reiniciarperfil():
    global user_id, dfusuarios

In [5]:
# Lista de géneros de películas
generos = ['Acción', 'Aventura', 'Animación', 'Comedia', 'Crimen', 'Documental',
           'Drama', 'Familiar', 'Fantasía', 'Historia', 'Horror', 'Musical',
           'Misterio', 'Romance', 'Ciencia Ficción', 'Suspense', 'Guerra', 'Western']

# Obtener la información del usuario
user = dfusuarios[dfusuarios['ID_Usuario'] == user_id].iloc[0]

In [6]:
# Vista Géneros Deseados

# Diccionario para seguir el estado de cada género y su botón asociado
estado_generos_deseados = {genero: None for genero in generos}
botones_deseados = {genero: widgets.Button(description=genero, layout=widgets.Layout(width='97%')) for genero in generos}

output_deseados1 = widgets.Output()
with output_deseados1:
    print("Selecciona hasta 5 géneros en el orden que más te gusten")

# Configurar el GridBox para acomodar los botones
grid_deseados = widgets.GridBox(list(botones_deseados.values()), layout=widgets.Layout(grid_template_columns="repeat(3, 154px)"))
#display(grid)

# Área de salida para mostrar los géneros seleccionados y mensajes
output_deseados2 = widgets.Output()
#display(output)

# Botón de continuar
btn_continuar1 = widgets.Button(description="Continuar", disabled=True)
btn_continuar1.on_click(lambda b: continuar1())
#display(btn_continuar)

for boton in botones_deseados.values():
    boton.on_click(boton_clicado1)

# Grupo de widgets en un VBox
vista_deseados = widgets.VBox([output_deseados1, grid_deseados, output_deseados2, btn_continuar1])

In [7]:
# Vista Géneros No Deseados

# Diccionario para seguir el estado de cada género no deseado y su botón asociado
estado_generos_no_deseados = {genero: None for genero in generos}
botones_no_deseados = {genero: widgets.Button(description=genero, layout=widgets.Layout(width='97%')) for genero in generos}

output_no_deseados1 = widgets.Output()
with output_no_deseados1:
    print("Selecciona hasta 5 géneros que menos te gusten")

grid_no_deseados = widgets.GridBox(list(botones_no_deseados.values()), layout=widgets.Layout(grid_template_columns="repeat(3, 154px)"))

output_no_deseados2 = widgets.Output()

btn_continuar2 = widgets.Button(description="Continuar", disabled=True)
btn_continuar2.on_click(lambda b: continuar2())

for boton in botones_no_deseados.values():
    boton.on_click(boton_clicado2)

vista_no_deseados = widgets.VBox([output_no_deseados1, grid_no_deseados, output_no_deseados2, btn_continuar2])

#display(vista_no_deseados)
#inicializar_botones_no_deseados()

In [8]:
output_vista = widgets.Output()

display(output_vista)

reiniciarperfil()

MostrarVistaGenDeseados()


Output()

In [9]:
#dfusuarios.head(15)